# Fair Division of Indivisible Goods

This is Jupyter Notebook coming along with the chapter 
* (Handbook of Computational Social Choice)


## Defining MARA problems

In [1]:
from problem import Problem

agent 0{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 1{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 2{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 3{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}

agent 0{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 1{'r1': 2, 'r2': 5, 'r5': 2, 'r3': 3, 'r4': 7, 'r0': 1}
agent 2{'r1': 6, 'r2': 8, 'r5': 2, 'r3': 1, 'r4': 1, 'r0': 2}
agent 3{'r1': 4, 'r2': 4, 'r5': 2, 'r3': 3, 'r4': 2, 'r0': 5}



A problem is defined by specifying :
* a number of agents
* a number of resources
* a way to generate utilities
* an initial allocation of resources
* a visibility graph

In [3]:
p1 = Problem(4,6,'empty','auctioneer','complete')
print (p1)
print (p1.printAllocation())

agent 0{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 1{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 2{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 3{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0['r1', 'r2', 'r5', 'r3', 'r4', 'r0']	 0
agent  1                                 []	 0
agent  2                                 []	 0
agent  3                                 []	 0



In [5]:
p1.setUtilities(
[{'r0':0,'r1':0,'r2':0,'r3':0,'r4':0,'r5':0},\
{'r0':1,'r1':2,'r2':5,'r3':3,'r4':7,'r5':2},\
{'r0':2,'r1':6,'r2':8,'r3':1,'r4':1,'r5':2},\
{'r0':5,'r1':4,'r2':4,'r3':3,'r4':2,'r5':2}]
)
print(p1)

agent 0{'r1': 0, 'r2': 0, 'r5': 0, 'r3': 0, 'r4': 0, 'r0': 0}
agent 1{'r1': 2, 'r2': 5, 'r5': 2, 'r3': 3, 'r4': 7, 'r0': 1}
agent 2{'r1': 6, 'r2': 8, 'r5': 2, 'r3': 1, 'r4': 1, 'r0': 2}
agent 3{'r1': 4, 'r2': 4, 'r5': 2, 'r3': 3, 'r4': 2, 'r0': 5}



## Solving via MIP

In [6]:
import mipsolving

In [9]:
print(mipsolving.envyminimizingLP(p1,verbose=True))

agent  1  gets resource r3
agent  1  gets resource r4
agent  2  gets resource r2
agent  2  gets resource r5
agent  3  gets resource r0
agent  3  gets resource r1
0.0


In [11]:
print(mipsolving.assignmentLP(p1))

agent  1  gets resource r3
agent  1  gets resource r4
agent  2  gets resource r2
agent  2  gets resource r5
agent  3  gets resource r0
agent  3  gets resource r1
9.0


## Protocols

### Picking Sequences

In [1]:
import protocols

agent 0{'r4': 1, 'r0': 4, 'r5': 6, 'r2': 5, 'r3': 3, 'r1': 2}
agent 1{'r2': 4, 'r1': 1, 'r5': 2, 'r4': 3, 'r3': 6, 'r0': 5}
agent 2{'r2': 3, 'r5': 2, 'r4': 6, 'r3': 4, 'r1': 5, 'r0': 1}
agent 3{'r5': 1, 'r3': 2, 'r2': 4, 'r4': 3, 'r1': 5, 'r0': 6}

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0['r4', 'r0', 'r5', 'r2', 'r3', 'r1']	21
agent  1                                 []	 0
agent  2                                 []	 0
agent  3                                 []	 0

agent  1  picks  r3
agent  2  picks  r4
agent  3  picks  r0
agent  3  picks  r1
agent  2  picks  r2
agent  1  picks  r5
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0                                 []	 0
agent  1                       ['r3', 'r5']	 8
agent  2                       ['r4', 'r2']	 9
agent  3                       ['r0', 'r1']	11

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
